In [1]:
import math

import torch
from tensordict import TensorDict

from fabricrl.data.buffers import ReplayBuffer

In [2]:
buf_size = 16
seq_len = 2
n_envs = 3
rb = ReplayBuffer(buf_size, n_envs)

In [3]:
data = TensorDict(
    {
        "observations": torch.rand(seq_len, n_envs, 4),
        "actions": torch.randint(0, 4, (seq_len, n_envs, 1)),
        "rewards": torch.rand(seq_len, n_envs, 1),
        "dones": torch.randint(0, 2, (seq_len, n_envs, 1)),
    },
    batch_size=[seq_len, n_envs],
)
data.shape, rb._buf.shape

(torch.Size([2, 3]), torch.Size([16, 3]))

In [4]:
while not rb._full:
    rb.add(data)

In [5]:
rb.sample(4)

TensorDict(
    fields={
        actions: Tensor(shape=torch.Size([4, 3, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        dones: Tensor(shape=torch.Size([4, 3, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        next_obs: Tensor(shape=torch.Size([4, 3, 4]), device=cpu, dtype=torch.float32, is_shared=False),
        observations: Tensor(shape=torch.Size([4, 3, 4]), device=cpu, dtype=torch.float32, is_shared=False),
        rewards: Tensor(shape=torch.Size([4, 3, 1]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([4, 3]),
    device=None,
    is_shared=False)

In [6]:
rb.buffer.view(math.prod(rb.buffer.shape), -1)

_ViewedTensorDict(
	source=TensorDict(
	    fields={
	        actions: Tensor(shape=torch.Size([16, 3, 1]), device=cpu, dtype=torch.int64, is_shared=False),
	        dones: Tensor(shape=torch.Size([16, 3, 1]), device=cpu, dtype=torch.int64, is_shared=False),
	        observations: Tensor(shape=torch.Size([16, 3, 4]), device=cpu, dtype=torch.float32, is_shared=False),
	        rewards: Tensor(shape=torch.Size([16, 3, 1]), device=cpu, dtype=torch.float32, is_shared=False)},
	    batch_size=torch.Size([16, 3]),
	    device=cpu,
	    is_shared=False), 
	op=view(size=torch.Size([48, 1])))

In [7]:
rb["returns"] = torch.rand(16, 3, 2)

In [8]:
rb.buffer.view(math.prod(rb.shape), -1)

_ViewedTensorDict(
	source=TensorDict(
	    fields={
	        actions: Tensor(shape=torch.Size([16, 3, 1]), device=cpu, dtype=torch.int64, is_shared=False),
	        dones: Tensor(shape=torch.Size([16, 3, 1]), device=cpu, dtype=torch.int64, is_shared=False),
	        observations: Tensor(shape=torch.Size([16, 3, 4]), device=cpu, dtype=torch.float32, is_shared=False),
	        returns: Tensor(shape=torch.Size([16, 3, 2]), device=cpu, dtype=torch.float32, is_shared=False),
	        rewards: Tensor(shape=torch.Size([16, 3, 1]), device=cpu, dtype=torch.float32, is_shared=False)},
	    batch_size=torch.Size([16, 3]),
	    device=cpu,
	    is_shared=False), 
	op=view(size=torch.Size([48, 1])))